In [18]:
# Import required libraries
import pandas as pd
import dash
from dash import html 
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create list of unique sites
unique_sites=spacex_df['Launch Site'].unique()

# Create dropdown options from unique sites
dropdown_options=[{'label':site,'value':site} for site in unique_sites]

# Append All sites to dropdown options
dropdown_options.insert(0,{'label': 'All Sites', 'value': 'ALL'})

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites

                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=dropdown_options,
                                    value='ALL',
                                    placeholder='All Sites'),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, 
                                    max=10000, 
                                    step=1000,
                                    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                                    value=[0, 10000]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df

    if entered_site == 'ALL':
        # Count successes and failures for all sites
        success_counts = spacex_df['Launch Site'][spacex_df['class']==1].value_counts().reset_index()
        success_counts.columns = ['Launch Site', 'count']

        fig = px.pie(success_counts,
            values='count', 
            names='Launch Site', 
            title='Total Success Launches for All Sites')
        
    else:
        # Filter dataframe by the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]

        # Count success and failure for the specific site
        site_counts = filtered_df['class'].value_counts()

        fig = px.pie(values = site_counts,
            names=site_counts.index,
            title=f'Success vs Failure Launches for {entered_site}')
    return fig
        # return the outcomes piechart for a selected site
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
             [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value')])

def update_scatter_plot(entered_site, payload_range):
    # Filter data by payload range
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) &
                            (spacex_df['Payload Mass (kg)'] <= high)]

    if entered_site == 'ALL':
        # Show data for all sites
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title='Payload vs. Success for ALL Sites',
            labels={'class': 'Launch Succes'}
        )
        
    else:
        # Filter data for selected site
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title=f'Payload vs. Success for {entered_site}',
            labels={'class': 'Launch Success'}
        )

    return fig


# python3.11 spacex_dash_app.py
# Run the app
if __name__ == '__main__':
    app.run_server()
